In [3]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_sample_weight
import pickle
import numpy as np
np.set_printoptions (threshold=None, edgeitems=None, linewidth=None, suppress=None)

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
df = pd.read_csv("/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_4jun_600_win_600twin.csv")

/tmp/ipykernel_1552358/222104805.py:1: DtypeWarning: Columns (63,68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_4jun_600_win_600twin.csv")


In [6]:
df.head()

,Unnamed: 0,step,rolling_junction_mean_speed_0,rolling_traffic_count_0,rolling_traffic_occupancy_0,rolling_junction_mean_speed_1,rolling_traffic_count_1,rolling_traffic_occupancy_1,rolling_junction_mean_speed_2,rolling_traffic_count_2,rolling_traffic_occupancy_2,rolling_junction_mean_speed_3,rolling_traffic_count_3,rolling_traffic_occupancy_3,rolling_junction_mean_speed_4,rolling_traffic_count_4,rolling_traffic_occupancy_4,rolling_junction_mean_speed_5,rolling_traffic_count_5,rolling_traffic_occupancy_5,rolling_junction_mean_speed_6,rolling_traffic_count_6,rolling_traffic_occupancy_6,rolling_junction_mean_speed_7,rolling_traffic_count_7,rolling_traffic_occupancy_7,rolling_junction_mean_speed_8,rolling_traffic_count_8,rolling_traffic_occupancy_8,rolling_junction_mean_speed_9,rolling_traffic_count_9,rolling_traffic_occupancy_9,rolling_junction_mean_speed_10,rolling_traffic_count_10,rolling_traffic_occupancy_10,rolling_junction_mean_speed_11,rolling_traffic_count_11,rolling_traffic_occupancy_11,rolling_junction_mean_speed_12,rolling_traffic_count_12,rolling_traffic_occupancy_12,rolling_junction_mean_speed_13,rolling_traffic_count_13,rolling_traffic_occupancy_13,rolling_junction_mean_speed_14,rolling_traffic_count_14,rolling_traffic_occupancy_14,rolling_junction_mean_speed_15,rolling_traffic_count_15,rolling_traffic_occupancy_15,rolling_junction_mean_speed_16,rolling_traffic_count_16,rolling_traffic_occupancy_16,rolling_junction_mean_speed_17,rolling_traffic_count_17,rolling_traffic_occupancy_17,rolling_junction_mean_speed_18,rolling_traffic_count_18,rolling_traffic_occupancy_18,rolling_junction_mean_speed_19,rolling_traffic_count_19,rolling_traffic_occupancy_19,time_of_day,incident_edge,incident_start_time,incident_type,accident_id,accident_duration,incident_lane,accident_label,rolling_travel_time_3_22,rolling_travel_time_21_4,rolling_travel_time_1_16,rolling_travel_time_15_2,rolling_travel_time_40_20,rolling_travel_time_19_41,rolling_travel_time_42_18,rolling_travel_time_17_39,rolling_travel_time_3_41,rolling_travel_time_42_31,rolling_travel_time_30_39,rolling_travel_time_40_4,rolling_travel_time_32_2,rolling_travel_time_1_29,rolling_travel_time_32_16,rolling_travel_time_15_29,rolling_travel_time_30_18,rolling_travel_time_17_31
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,NaN,NaN,0.0,-1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0.0,NaN,NaN,0.0,-1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0.0,NaN,NaN,0.0,-1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,0.0,NaN,NaN,0.0,-1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0.0,NaN,NaN,0.0,-1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [7]:
df.isna().sum()

Unnamed: 0                              0
step                                    0
rolling_junction_mean_speed_0         599
rolling_traffic_count_0               599
rolling_traffic_occupancy_0           599
rolling_junction_mean_speed_1         599
rolling_traffic_count_1               599
rolling_traffic_occupancy_1           599
rolling_junction_mean_speed_2         599
rolling_traffic_count_2               599
rolling_traffic_occupancy_2           599
rolling_junction_mean_speed_3         599
rolling_traffic_count_3               599
rolling_traffic_occupancy_3           599
rolling_junction_mean_speed_4         599
rolling_traffic_count_4               599
rolling_traffic_occupancy_4           599
rolling_junction_mean_speed_5         599
rolling_traffic_count_5               599
rolling_traffic_occupancy_5           599
rolling_junction_mean_speed_6         599
rolling_traffic_count_6               599
rolling_traffic_occupancy_6           599
rolling_junction_mean_speed_7     

In [8]:
road_name_edge_id = {
1:{"934465920","5614812#0","889439250","436794672#0","1078715158"},
2:{"532215357#0","436794668#0","436794668#7","436794677#0","436794673#0"},
3:{"436791116","436791119#0","436791122#0","436791121#0","436791111"},
4:{"1070423862#0","5602753#1","5602753#2","436790493#0","533573789#0","533573789#2","436790492#0","436790484","512811687#0"},
5:{"436942365#0","436942382#0","436942369#0","436942367","436942384#0","436942364","436942372","395215600","966303717","436942386#0"},
6:{"532227836","532227834#0","436789544#0","436789576#0","436789539#0","436789539#2","436789539#7","436789570#0"},
7:{"-436942361#7","-436942361#3","-436942361#1","-436942358#5","-436942358#3","-436942358#0","345713658#0"},
8:{"-436789319","395490730","1051025192"},
9:{"436940273#0","692089619#0","692089616#0","692089616#2","436940272#0","436940271#0","5635238","692089613#0","692089613#2","692089613#6","692089611#0"},
10:{"512810351#0","436943782","436943780","436943781"},
11:{"436943721","436943716","436943727#0","406379830#0","436943736","436943731#0","436943728","436943723","436943726","436943720","436943747#0","436943754#0","436943740","436943735","436943729","436943741#0","436943752#0"},
12:{"-436943756#2","-911576955#2","909831620","911576960","-327757100#6"}

}

In [9]:
def standardize(col):
   return (col - col.mean()) / col.std()

In [10]:
# IF Localizing

# df = df[df["accident_label"] == True]
# df['label'] = None
# df =df.fillna(-1)

# for category, edges in road_name_edge_id.items():
# # Find rows where incident_edge is in the current set of edges
#     mask = df['incident_edge'].isin(edges)
# # Set the appropriate label column to 1 for these rows
#     df.loc[mask, 'label'] = category-1  # Adjusted index by -1 for zero-based indexing

# Y = df["label"]


    
# X = df.drop(["incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label","label"],axis=1)
# X = X.apply(standardize)

# X = X.drop(["Unnamed: 0","step"],axis=1)

In [11]:
# IF detecting incidents
df = df[600:]
df =df.fillna(-1)

Y = df["accident_label"]


    
X = df.drop(["Unnamed: 0","step","incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label"],axis=1)
X = X.apply(standardize)

In [12]:

X= X.values
Y= Y.values

print("Splitting into test train")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

Splitting into test train


In [13]:
X_train.shape

(2073120, 79)

In [14]:
clf = TabNetClassifier(device_name='cuda')

/home/local/ASURITE/speddira/miniconda3/envs/cobra/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [15]:
clf.fit(
  X_train, y_train,num_workers = 10,max_epochs=80
)

/home/local/ASURITE/speddira/miniconda3/envs/cobra/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.53682 |  0:01:06s
epoch 1  | loss: 0.39614 |  0:02:14s
epoch 2  | loss: 0.346   |  0:03:25s
epoch 3  | loss: 0.32916 |  0:04:40s
epoch 4  | loss: 0.3189  |  0:05:52s
epoch 5  | loss: 0.31225 |  0:07:05s
epoch 6  | loss: 0.30861 |  0:08:19s
epoch 7  | loss: 0.30381 |  0:09:31s
epoch 8  | loss: 0.30072 |  0:10:47s
epoch 9  | loss: 0.29669 |  0:12:10s
epoch 10 | loss: 0.29568 |  0:13:29s
epoch 11 | loss: 0.29353 |  0:14:41s
epoch 12 | loss: 0.29094 |  0:15:50s
epoch 13 | loss: 0.28978 |  0:17:03s
epoch 14 | loss: 0.28775 |  0:18:13s
epoch 15 | loss: 0.28803 |  0:19:36s
epoch 16 | loss: 0.28885 |  0:20:50s
epoch 17 | loss: 0.29185 |  0:22:12s
epoch 18 | loss: 0.2868  |  0:23:28s
epoch 19 | loss: 0.28533 |  0:24:45s
epoch 20 | loss: 0.28605 |  0:25:57s
epoch 21 | loss: 0.28865 |  0:27:14s
epoch 22 | loss: 0.2882  |  0:28:26s
epoch 23 | loss: 0.28406 |  0:29:40s
epoch 24 | loss: 0.28566 |  0:30:48s
epoch 25 | loss: 0.28554 |  0:32:04s
epoch 26 | loss: 0.28437 |  0:33:21s
e

In [16]:
saving_path_name = "tab_net_4_jun"
saved_filepath = clf.save_model(saving_path_name)

Successfully saved model at tab_net_4_jun.zip


In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

X= X_test
Y= y_test
y_pred = clf.predict(X)
y_pred = list(y_pred)
y_test = list(Y)

accuracy = accuracy_score(y_test, y_pred)*100
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy))
print(f"FAR {100-accuracy}")
print("precision: %.2f%%" % (precision * 100.0))
print("f1: %.2f%%" % (f1 * 100.0))
print("recall: %.2f%%" % (recall * 100.0))

Accuracy: 94.97%
FAR 5.034730261634635
precision: 94.93%
f1: 94.02%
recall: 93.12%
